In [ ]:
%matplotlib qt

import numpy as np
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
from matplotlib import animation

In [ ]:
ARBITRARY_SWAP = True
CONSECUTIVE_SWAP = False

def distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

    
def tsp_annealing(points, Tmax = None, Tmin = None, Tchange = lambda T: 0.99*T, mode=ARBITRARY_SWAP):
    n = len(points)
    
    distances = {(points[i][0], points[i][1], points[j][0], points[j][1]): distance(points[i], points[j]) for j in range(n) for i in range(n)}
    
    def path_energy(path):
        E = 0
        n = len(path)
        for i in range(n):
            j = (i + 1) % n
            E += distances[path[i][0], path[i][1], path[j][0], path[j][1]]
        return E
    
    x_min = np.min(points[:, 0])
    x_max = np.max(points[:, 0])
    
    y_min = np.min(points[:, 1])
    y_max = np.max(points[:, 1])
    
    max_dist = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)
    
    if Tmax is None:
        Tmax = 2 * n * max_dist / np.log(2)
        
    if Tmin is None:
        Tmin = max_dist / (n * 20 * np.log(20))
    
    current_path = points
    current_energy = path_energy(current_path)
    energies = [current_energy]
    paths = [current_path]
    T = Tmax
    while T > Tmin:
        for _ in range(100):
            start_point = np.random.randint(0, n)
            
            if mode is ARBITRARY_SWAP:
                end_point = start_point + np.random.randint(0, n)

                if end_point >= n:
                    (start_point, end_point) = (end_point - n, start_point)
                    
                new_path = current_path.copy()
                new_path[start_point : end_point + 1] = new_path[start_point : end_point + 1][::-1] 
                new_energy = path_energy(new_path)
                
            else:
                new_path = current_path.copy()
                new_path[start_point-1:start_point+1] = new_path[start_point-1:start_point+1][::-1] 
                new_energy = path_energy(new_path)

            delta_energy = new_energy - current_energy
            if delta_energy < 0 or bernoulli.rvs(p=np.exp(-delta_energy/T)) == 1:
                current_path = new_path
                current_energy = new_energy
        
        paths.append(current_path)
        energies.append(current_energy)
        T = Tchange(T)

    return (np.array(paths), energies)

In [ ]:
def show_solution(path):
    path = np.vstack((path, [path[0]]))

    fig = plt.figure()
    ax1 = fig.add_subplot(1,1,1)

    plt.plot(path[:, 0], path[:, 1])
    plt.show()

In [ ]:
def animate_tsp(found_paths, found_lengths):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    n = len(found_lengths)
    
    paths_data = found_paths[0::10]
    lengths_data = found_lengths[0::10]
    
    if n % 10 != 0:
        paths_data = np.append(paths_data, [found_paths[-1]], axis=0)
        lengths_data = np.append(lengths_data, [found_lengths[-1]], axis=0)
    
    def animate(i):
        data = paths_data[i]

        data = np.append(data, [data[0]], axis=0)
        
        ax.clear()
        ax.plot(data[:, 0], data[:, 1])

        iteration = i*10 if i*10 < n else n-1
        
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title(f"Iteration {iteration}: length: {np.round(lengths_data[i], 2)}")
        
    return animation.FuncAnimation(fig, animate, frames=len(lengths_data), interval=2)

# Przykładowe dane

In [ ]:
uniform_10 = np.random.uniform(0, 10, size=(10,2))
uniform_50 = np.random.uniform(0, 10, size=(50,2))
uniform_100 = np.random.uniform(0, 10, size=(100,2))

uni_arr = [uniform_10, uniform_50, uniform_100]

normal_20 = np.vstack(list(np.random.normal([np.random.uniform(0,40), np.random.uniform(0,40)], i/2+1, size=(5,2)) for i in range(4)))
normal_60 = np.vstack(list(np.random.normal([np.random.uniform(0,40), np.random.uniform(0,40)], i/2+1, size=(15,2)) for i in range(4)))
normal_100 = np.vstack(list(np.random.normal([np.random.uniform(0,40), np.random.uniform(0,40)], i/2+1, size=(25,2)) for i in range(4)))

norm_arr = [normal_10, normal_50, normal_100]

from itertools import product

separated_27 = np.vstack(list(np.random.uniform([10*i, 10*j], [10*i + 5, 10*j + 5], size=(3, 2)) for i, j in product(range(3),repeat=2)))
separated_90 = np.vstack(list(np.random.uniform([10*i, 10*j], [10*i + 5, 10*j + 5], size=(10, 2)) for i, j in product(range(3),repeat=2)))
separated_270 = np.vstack(list(np.random.uniform([10*i, 10*j], [10*i + 5, 10*j + 5], size=(30, 2)) for i, j in product(range(3),repeat=2)))

plt.plot(normal_100[:,0], normal_100[:,1], 'ro')

sep_arr = [separated_27, separated_90, separated_270]

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

axes = [ax1, ax2, ax3, ax4]

for i, ex in enumerate(uni_arr):
    paths, _ = tsp_annealing(ex)
    path = paths[-1]
    path = np.vstack((path, [path[0]]))

    axes[i].plot(path[:, 0], path[:, 1])

plt.show()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

axes = [ax1, ax2, ax3, ax4]

for i, ex in enumerate(norm_arr):
    paths, _ = tsp_annealing(ex)
    path = paths[-1]
    path = np.vstack((path, [path[0]]))

    axes[i].plot(path[:, 0], path[:, 1])

plt.show()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

axes = [ax1, ax2, ax3, ax4]

for i, ex in enumerate(sep_arr):
    paths, _ = tsp_annealing(ex)
    path = paths[-1]
    path = np.vstack((path, [path[0]]))

    axes[i].plot(path[:, 0], path[:, 1])

plt.show()

# Animacja pokazująca działanie

In [ ]:
paths, lens = tsp_annealing(uniform_50)
ani = animate_tsp(paths, lens)

# Zbieżność rozwiązania w zależności od funkcji zmiany temperatury

In [ ]:
_, lens1 = tsp_annealing(uniform_50, Tchange=lambda T: 0.8*T)
_, lens2 = tsp_annealing(uniform_50, Tchange=lambda T: 0.9*T)
_, lens3 = tsp_annealing(uniform_50, Tchange=lambda T: 0.95*T)
_, lens4 = tsp_annealing(uniform_50, Tchange=lambda T: 0.99*T)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

ax1.plot(lens1)
ax2.plot(lens2)
ax3.plot(lens3)
ax4.plot(lens4)
plt.show()

# Zbieżność rozwiązania w zależności od sposobu wyboru sąsiadów

In [ ]:
_, lens1 = tsp_annealing(uniform_50, mode=ARBITRARY_SWAP)
_, lens2 = tsp_annealing(uniform_50, mode=CONSECUTIVE_SWAP)

fig, (ax1, ax2) = plt.subplots(2)

ax1.plot(lens1)
ax2.plot(lens2)
plt.show()